In [ ]:
from transformers import AutoImageProcessor, Dinov2Model
import torch
from datasets import load_dataset

dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

image_processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = Dinov2Model.from_pretrained("facebook/dinov2-base")

inputs = image_processor([image,image], return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
list(last_hidden_states.shape)

In [ ]:
inputs.cuda()

In [ ]:
model.config._name_or_path

In [ ]:
import torch
import torch.nn as nn

class SiglipMLP(nn.Module):
    def __init__(self, input_dim, intermediate_dim, output_dim):
        super().__init__()
        self.pre_norm = nn.LayerNorm(input_dim)
        self.proj = nn.Sequential(
            nn.Linear(input_dim, intermediate_dim),
            nn.GELU(),
            nn.Linear(intermediate_dim, output_dim)
        )

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        hidden_states = self.pre_norm(hidden_states)
        hidden_states = hidden_states+self.proj(hidden_states)
        return hidden_states

class VLContrastHead(nn.Module):
    def __init__(self, vision_dimesion, text_dimension, device, target_dimension=512, linear=False):
        super(VLContrastHead, self).__init__()
        self.device = device if device else ('cuda' if torch.cuda.is_available() else 'cpu')
        self.linear = linear
        if self.linear:
            self.vision_mapping_network = nn.Linear(vision_dimesion, target_dimension)
            self.text_mapping_network = nn.Linear(text_dimension, target_dimension)
        else:
            # self.vision_mapping_network = SiglipMLP(vision_dimesion, target_dimension, target_dimension)
            # self.text_mapping_network = SiglipMLP(text_dimension, target_dimension, target_dimension)
            self.vision_mapping_network = nn.Linear(vision_dimesion, target_dimension)
            self.text_mapping_network = nn.Linear(text_dimension, target_dimension)
            self.mapping_network = SiglipMLP(target_dimension, target_dimension, target_dimension)

        self.vision_layer_norm = nn.LayerNorm(vision_dimesion)
        self.text_layer_norm = nn.LayerNorm(text_dimension)
        self.logit_scale = nn.Parameter(torch.randn(1))
        self.logit_bias = nn.Parameter(torch.randn(1))

        self._initialize_weights()
    
    def _initialize_weights(self):

        for module in self.modules():
            if isinstance(module, nn.Linear):
                torch.nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    torch.nn.init.zeros_(module.bias)
            elif isinstance(module, nn.LayerNorm):
                torch.nn.init.ones_(module.weight)
                torch.nn.init.zeros_(module.bias)

        # Initialize logit_scale and logit_bias
        logit_scale_init = torch.log(torch.tensor(10.0))
        self.logit_scale.data.fill_(logit_scale_init)
        self.logit_bias.data.fill_(torch.tensor(-10.0))

In [ ]:
head = VLContrastHead(512, 512, 'cuda')

In [ ]:
head.mapping_network.proj[2].bias

In [ ]:
import torch

# 尝试释放显存
torch.cuda.empty_cache()

# 查看显存使用情况
print(torch.cuda.memory_allocated(0))
print(torch.cuda.memory_reserved(0))


In [ ]:
import torch
import torch.nn as nn
from typing import Optional
class StarMLP(nn.Module):
    def __init__(
        self,
        input_dim: int,
        output_dim: int,
        intermediate_dim: Optional[int] = None,
    ):
        super().__init__()
        intermediate_dim = intermediate_dim if intermediate_dim is not None else output_dim
        self.Wa = nn.Linear(input_dim, input_dim, bias=False)
        self.Wb = nn.Linear(input_dim, input_dim, bias=False)
        self.g = nn.Linear(input_dim, output_dim, bias=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        a = self.Wa(x)  # N x d
        b = self.Wb(x)  # N x d
        x = torch.einsum('bij,bj->bi', torch.sigmoid(a.unsqueeze(-1) * b.unsqueeze(1)), x)
        x = self.g(x)

        assert not torch.isnan(x).any(), "Output contains NaN"
        assert not torch.isinf(x).any(), "Output contains infinite values"

        return x

In [ ]:
networ = StarMLP(64, 128)


In [ ]:
x = torch.randn(16, 64)
networ(x).shape

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

path = 'Alibaba-NLP/gte-base-en-v1.5'
device = torch.device('cuda')
tokenzier = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(
    path,
    trust_remote_code=True,
    unpad_inputs=True,
    use_memory_efficient_attention=True,
).to(device)




In [13]:
input_texts = [
    "what is the capital of China?",
    "how to implement quick sort in python?",
    "Beijing",
    "sorting algorithms"
]
inputs = tokenzier(input_texts, max_length=8192, padding=True, truncation=True, return_tensors='pt')
with torch.autocast(device_type=device.type, dtype=torch.float16):  # or bfloat16
    with torch.inference_mode():
        outputs = model(**inputs.to(device))

In [ ]:
outputs.last_hidden_state[:, 0]

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

model = AutoModel.from_pretrained("google/siglip-base-patch16-224")


In [ ]:
model.logit_scale.exp()

In [ ]:
import torch

def random_mask(self, v1, v2, r: float):
    """
    随机从每个向量中挑选 k 个维度，同时保证梯度传导
    :param v1: 第一个向量, 形状为 (n, d)
    :param v2: 第二个向量, 形状为 (n, d)
    :param r: 选择的维度比例（例如 0.5 表示选择一半的维度）
    :return: 选中的维度对应的子向量 v1 和 v2
    """
    assert v1.shape == v2.shape, "两个张量的形状必须相同"
    n, d = v1.shape
    k = int(d * r)
    
    # 获取 v1 所在设备
    device = v1.device
    
    # 在相同的设备上生成随机索引
    indices = torch.randperm(d, device=device)[:k]  # 从 d 中随机选择 k 个不重复的索引
    print(indices)
    # 选取对应维度
    selected_v1 = v1[:, indices]  # 使用高级索引选择维度
    selected_v2 = v2[:, indices]
    
    return selected_v1, selected_v2

# 示例代码
v1 = torch.randn(3, 5, device='cuda', requires_grad=True)  # 形状为 (5, 10) 的向量
v2 = torch.randn(3, 5, device='cuda', requires_grad=True)  # 形状为 (5, 10) 的向量
r = 0.6  # 选择50%的维度

selected_v1, selected_v2 = random_mask(None, v1, v2, r)
print("随机选择维度的结果:", selected_v1, selected_v2)


In [ ]:
selected_v1.shape

In [ ]:
import torch

# 假设你的张量形状为 (n, d)
n, d = 5, 4  # 示例数据
tensor = torch.randn(n, d)

# 定义分组数 m
m = 2  # 比如将 d 维度分成 2 组

# 检查 d 能否被 m 整除
assert d % m == 0, "d 维度必须能被 m 整除"

# 计算每组的大小
group_size = d // m

# 重塑张量，以便在 m 组上进行平均池化
tensor_reshaped = tensor.view(n, m, group_size)

# 对每组进行平均池化
pooled_result = tensor_reshaped.max(dim=-1)

print(pooled_result)


In [ ]:
tensor

In [ ]:
max_v, max_index = tensor[:, 2:4].max(dim=-1)
max_v

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

path = 'Alibaba-NLP/gte-Qwen2-1.5B-instruct'
device = torch.device('cuda')
tokenzier = AutoTokenizer.from_pretrained(path, trust_remote_code=True)
model = AutoModel.from_pretrained(
    path,
    trust_remote_code=True,
    unpad_inputs=True,
    use_memory_efficient_attention=True,
    torch_dtype=torch.float16
).to(device)

inputs = tokenzier(['test input'], truncation=True, max_length=8192, padding=True, return_tensors='pt')

with torch.inference_mode():
    outputs = model(**inputs.to(device))


In [ ]:
from transformers import TRANSFORMERS_CACHE
print(TRANSFORMERS_CACHE)


In [9]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests
import torch

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
device = torch.device('cuda')
image_processor = AutoImageProcessor.from_pretrained("facebook/vit-mae-large",use_fast=True)
model = AutoModel.from_pretrained("facebook/vit-mae-large", attn_implementation="sdpa", torch_dtype=torch.float16).to(device)

In [8]:
inputs = image_processor(images=[image]*1024, return_tensors="pt").to(model.device, dtype=torch.float16)
with torch.no_grad():
    outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state

In [1]:
from transformers import AutoImageProcessor, Dinov2Model
import torch
from datasets import load_dataset

# dataset = load_dataset("huggingface/cats-image", trust_remote_code=True)
# image = dataset["test"]["image"][0]
# device = torch.device('cuda')
image_processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
# model = Dinov2Model.from_pretrained("facebook/dinov2-base", torch_dtype=torch.float16).to(device)



In [ ]:
from PIL import Image   
image = Image.open('/home/mila/l/le.zhang/scratch/datasets/LAION/LAION30M/images/0000009/0007528.jpg')
image.size

In [ ]:
inputs = image_processor(image, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
list(last_hidden_states.shape)

In [1]:
import torch
weight = torch.load('/home/mila/l/le.zhang/scratch/light_align/logs/sharelock12m_NV2dinoL_bs_16384_lion_lr_1e-3_star7L_d4096_scale10_negbias10_clip_test/checkpoints/epoch_6.pt')

/tmp/ipykernel_2565747/4216871878.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weight = torch.load('/home/mila/l/le.zhang/scratch/light_align/logs/sharelock12m_NV2din

In [ ]:
print(weight['state_dict']['vision_mapping_network.f1.weight'].shape)
print(weight['state_dict']['vision_mapping_network.f2.weight'].shape)
print(weight['state_dict']['vision_mapping_network.g.weight'].shape)

print(weight['state_dict']['text_mapping_network.f1.weight'].shape)
print(weight['state_dict']['text_mapping_network.f2.weight'].shape)
print(weight['state_dict']['text_mapping_network.g.weight'].shape)

In [ ]:
import torch

def z_score_normalize(features):
    mean = features.mean(dim=0, keepdim=True)
    std = features.std(dim=0, keepdim=True)
    return (features - mean) / std

features = torch.randn(10, 1024)
features = z_score_normalize(features)
print(features)
print((features - features.mean(0)) / features.std(0))

In [ ]:
class ShareLockMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.2):
        super(ShareLockMLP self).__init__()
        
        # Define the layers
        self.layers = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_prob),

            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_prob),

            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_prob),

            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.layers(x)

mlp = ShareLockMLP(1024, 4096, 2048)

In [1]:
from transformers import AutoTokenizer, CLIPTextModel

model = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", device_map='cuda').half()
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-large-patch14")

inputs = tokenizer(["a photo of a cat", "a photo of a dog"], padding=True, return_tensors="pt").to('cuda')

outputs = model(**inputs)
last_hidden_state = outputs.last_hidden_state
pooled_output = outputs.pooler_output  # pooled (EOS token) states

/home/mila/l/le.zhang/.conda/envs/openflamingo/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/mila/l/le.zhang/.conda/envs/openflamingo/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Could not load library libnvrtc.so.12. Error: libnvrtc.so.12: cannot open shared object file: No such file or directory
Could not load library libnvrtc.so. Error: libnvrtc.so: cannot open shared object file: No such file or directory
Could not load library libnvrtc.so.12. Error: libnvrtc.so.12: cannot open shared object file: No such file or directory
Could not load library libnvrtc.so. Error: libnvrtc.so: cannot open shared object fi

RuntimeError: cuDNN Frontend error: [cudnn_frontend] Error: No execution plans support the graph.